In [343]:
import pandas as pd

In [344]:
land_units =        pd.read_csv(r"..\data\raw\land_units_tables.tsv",           sep="\t", dtype=str, keep_default_na=False)
main_units =        pd.read_csv(r"..\data\raw\main_units_tables.tsv",           sep="\t", dtype=str, keep_default_na=False)
battle_entities =   pd.read_csv(r"..\data\raw\battle_entities_tables.tsv",      sep="\t", dtype=str, keep_default_na=False)
melee_weapons =     pd.read_csv(r"..\data\raw\melee_weapons_tables.tsv",        sep="\t", dtype=str, keep_default_na=False)
missile_weapons =   pd.read_csv(r"..\data\raw\missile_weapons_tables.tsv",      sep="\t", dtype=str, keep_default_na=False)
projectiles =       pd.read_csv(r"..\data\raw\projectiles_tables.tsv",          sep="\t", dtype=str, keep_default_na=False)
shields =           pd.read_csv(r"..\data\raw\unit_shield_types_tables.tsv",    sep="\t", dtype=str, keep_default_na=False)
armour_types =      pd.read_csv(r"..\data\raw\unit_armour_types_tables.tsv",    sep="\t", dtype=str, keep_default_na=False)

In [345]:
land_units = land_units[~land_units["key"].str.startswith("#", na=False)].copy()
main_units = main_units[~main_units["unit"].str.startswith("#", na=False)].copy()
battle_entities = battle_entities[~battle_entities["key"].str.startswith("#", na=False)].copy()
#melee_weapons = melee_weapons[~melee_weapons["placeholder_key"].str.startswith("#", na=False)].copy() 

In [346]:
land_unit_col = land_units[["key", "man_entity", "armour", "charge_bonus", "melee_attack", "melee_defence", "morale", "primary_melee_weapon"]].copy()
main_unit_col = main_units[["unit", "caste", "land_unit", "num_men"]].copy() 
battle_entities_col = battle_entities[["key", "mass", "hit_points", "size"]].copy()
melee_weapons_col = melee_weapons[["key", "bonus_v_large", "damage", "ap_damage", "melee_weapon_type", "melee_attack_interval"]].copy()
armour_types_col = armour_types[["key", "armour_value"]].copy()
shields_col = shields[["key", "missile_block_chance"]].copy()
projectiles_col = projectiles[["key", "effective_range", "damage", "ap_damage", "base_reload_time", "bonus_v_infantry", "bonus_v_large" ]].copy()
#missile_weapons_col = missile_weapons[["key", "default_projectile"]] How does this connect?



In [347]:
merged = main_unit_col.merge(
    land_unit_col,
    left_on="land_unit",
    right_on="key",
    how="left"
)

print("After merge 1:", merged.columns.tolist()) # Output check

merged = merged.merge(
    battle_entities_col,
    left_on="man_entity",
    right_on="key",
    how="left"
)

print("After merge 2:", merged.columns.tolist()) # Output check

merged = merged.merge(
    melee_weapons_col,
    left_on="primary_melee_weapon",
    right_on="key",
    how="left"
)



merged = merged.drop(columns=["key_x", "key_y", "key"], errors="ignore") # duplicate keys not allowed by pd mmmmm

print("After merge 3:", merged.columns.tolist()) # Output check

merged = merged.merge(
    armour_types_col,
    left_on="armour",
    right_on="key",
    how="left"
)

merged = merged.drop(columns=["key_x", "key_y", "key"], errors="ignore")
print("After merge 4:", merged.columns.tolist()) # Output check 




After merge 1: ['unit', 'caste', 'land_unit', 'num_men', 'key', 'man_entity', 'armour', 'charge_bonus', 'melee_attack', 'melee_defence', 'morale', 'primary_melee_weapon']
After merge 2: ['unit', 'caste', 'land_unit', 'num_men', 'key_x', 'man_entity', 'armour', 'charge_bonus', 'melee_attack', 'melee_defence', 'morale', 'primary_melee_weapon', 'key_y', 'mass', 'hit_points', 'size']
After merge 3: ['unit', 'caste', 'land_unit', 'num_men', 'man_entity', 'armour', 'charge_bonus', 'melee_attack', 'melee_defence', 'morale', 'primary_melee_weapon', 'mass', 'hit_points', 'size', 'bonus_v_large', 'damage', 'ap_damage', 'melee_weapon_type', 'melee_attack_interval']
After merge 4: ['unit', 'caste', 'land_unit', 'num_men', 'man_entity', 'armour', 'charge_bonus', 'melee_attack', 'melee_defence', 'morale', 'primary_melee_weapon', 'mass', 'hit_points', 'size', 'bonus_v_large', 'damage', 'ap_damage', 'melee_weapon_type', 'melee_attack_interval', 'armour_value']


In [348]:
(merged["unit"] == merged["land_unit"]).mean() # Zijn keys identiek? Phew 

np.float64(0.9341040462427745)

In [349]:
merged.to_csv("../data/clean/main_and_land_stats.csv", index=False)

In [350]:
import os
path = r"..\data\clean\main_and_land_stats.csv"
print(os.path.abspath(path))
merged.to_csv(path, index=False)


c:\Users\Jouke\Documents\wh3_da\data\clean\main_and_land_stats.csv


In [351]:
import os, time
print("mtime:", time.ctime(os.path.getmtime(path)))
print("size:", os.path.getsize(path))


mtime: Mon Feb  2 18:24:03 2026
size: 597669
